# Density based clustering the Facebook graph
Kishan Ved

22110122

### Import the dataframe

In [727]:
import pandas as pd
import numpy as np

df = pd.read_csv('musae_facebook_target.csv');
df

,id,facebook_id,page_name,page_type
0,0,145647315578475,The Voice of China 中国好声音,tvshow
1,1,191483281412,U.S. Consulate General Mumbai,government
2,2,144761358898518,ESET,company
3,3,568700043198473,Consulate General of Switzerland in Montreal,government
4,4,1408935539376139,Mark Bailey MP - Labor for Miller,politician
...,...,...,...,...
22465,22465,1379955382222841,Kurt Wiegel MdL,politician
22466,22466,1651527995097082,dubdub Stories,company
22467,22467,155369444540412,Ministerio del Interior - Paraguay,government
22468,22468,175067819212798,Tottus Perú,company


### Import the edges

In [728]:
edges_df = pd.read_csv("musae_facebook_edges.csv")
edges_df

,id_1,id_2
0,0,18427
1,1,21708
2,1,22208
3,1,22171
4,1,6829
...,...,...
170997,20188,20188
170998,22340,22383
170999,22348,22348
171000,5563,5563


### 2-Approx algorithm

1. tot - deg[v] = e(s,s)  {deg is a dict containing degrees}

2. Remove v from deg

3. Update deg of vertices that were connected to v

4. Find the density 

5. Store the vertex removed and the density of the subgraph obtained after this

### Dictionaries of degree, neighbors and self-loops (if >0) of vertices

In [729]:
deg = {}
neighbors = {}
self_loops = {}

for _, row in edges_df.iterrows():
    if row['id_1'] == row['id_2']:
        if row['id_1'] in self_loops:
            self_loops[row['id_1']] += 1
        else:
            self_loops[row['id_1']] = 1

    for node in ['id_1', 'id_2']:
        if row[node] in deg:
            deg[row[node]] += 1
        else:
            deg[row[node]] = 1

    # Append to the neighbors list
    if row['id_1'] in neighbors:
        neighbors[row['id_1']].append(row['id_2'])
    else:
        neighbors[row['id_1']] = [row['id_2']]

    if row['id_2'] in neighbors:
        neighbors[row['id_2']].append(row['id_1'])
    else:
        neighbors[row['id_2']] = [row['id_1']]


### Total edges in the graph

In [730]:
tot = sum(deg.values())/2
tot # e(S,S)

171002.0

### Total vertices in the graph

In [731]:
ctr = len(deg) # |S|
total_vertices = ctr
total_vertices

22470

### Arrays of density and vertex

In [732]:
density = [] # stores densities
vertex = [] # vertex[0:i] (i included) contains vertices removed to achive density[i+1], i+1 as density[0] is for the full graph

### Density of the full graph

In [733]:
density.append(tot/total_vertices) # Density of the full graph
print(density)

[7.610235870048954]


### Function to calucate the density of the subgraph after removing the min. degree vertex

In [734]:
def find_density_greedy():
    global tot
    global deg
    global ctr
    global edges_df
    global vertex
    
    v = min(deg, key=lambda k: deg[k]) # vertex with minimum degree
    vertex.append(v) # Store the removed vertex
    ctr = ctr - 1 # decrease |S| by 1 (as v is removed)
    tot = tot - deg[v] # Update e(S,S)
    # Check for self edges, in that case, we subtracted 2, but we must subtract only one
    if v in self_loops:
        tot += self_loops[v]
    
    den = tot / ctr
    density.append(den)
    deg.pop(v) # Remove v from deg
    
    selected_rows = pd.Series(neighbors[v])
    deg_update = selected_rows.value_counts().to_dict()
    for ver, count in deg_update.items():
        if ver in deg:
            deg[ver] -= count

In [735]:
i = 1
while(ctr>1):
    i+=1
    if (i%1000==0): # Prints i to get an estimate of how much execution is done
        print(i)
    find_density_greedy()
    if(tot != sum(deg.values())/2): # Checks for correctness
        print("here")

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000


In [736]:
density = np.array(density)

In [737]:
x = np.argmax(density) - 1 # -1 as density[0] is the density of the full graph.
density[x]

35.006211180124225

In [738]:
graph = []
for i in range(0, total_vertices):
    if i not in vertex[:x+1]:
        graph.append(i)
len(graph)

321

In [739]:
fullgraph = {}
for ver in range(total_vertices):
    type = df.loc[ver, 'page_type']
    if(type in fullgraph.keys()):
        fullgraph[type] += 1
    else:
        fullgraph[type] = 1

fullgraph

{'tvshow': 3327, 'government': 6880, 'company': 6495, 'politician': 5768}

In [740]:
dense1 = {}
for ver in graph:
    type = df.loc[ver, 'page_type']
    if(type in dense1.keys()):
        dense1[type] += 1
    else:
        dense1[type] = 1

dense1

{'tvshow': 60, 'company': 4, 'government': 257}

In [741]:
density[np.argmin(density)]

0.0

In [742]:
deg

{7606: 0}

In [743]:
df

,id,facebook_id,page_name,page_type
0,0,145647315578475,The Voice of China 中国好声音,tvshow
1,1,191483281412,U.S. Consulate General Mumbai,government
2,2,144761358898518,ESET,company
3,3,568700043198473,Consulate General of Switzerland in Montreal,government
4,4,1408935539376139,Mark Bailey MP - Labor for Miller,politician
...,...,...,...,...
22465,22465,1379955382222841,Kurt Wiegel MdL,politician
22466,22466,1651527995097082,dubdub Stories,company
22467,22467,155369444540412,Ministerio del Interior - Paraguay,government
22468,22468,175067819212798,Tottus Perú,company


In [744]:
len(graph)

321

In [745]:
df[df['id'].isin(graph)]


,id,facebook_id,page_name,page_type
44,44,1507698529534072,APB FOX,tvshow
127,127,354595427910013,GovX,company
159,159,167394579972573,JRTC and Fort Polk,government
568,568,127428534470290,The Orville,tvshow
679,679,146451038734267,Michigan National Guard,government
...,...,...,...,...
22252,22252,174788155866586,U.S. Army School of Music,government
22265,22265,61575637587,U.S. Pacific Command,government
22328,22328,114802901909136,North Carolina National Guard,government
22349,22349,110318209007293,21st Theater Sustainment Command,government
